In [281]:
!pip install simpy
import simpy
import random
import math
import numpy as np

In [282]:
Total_Time = 100

RANDOM_SEED = 1
random.seed(RANDOM_SEED)

hospital_treating_time_rate = 6
home_treating_time_rate = 10

num_of_individuals = 0
S = 147 + 177 + 207
if(S > 1000): num_of_individuals = S
elif(S > 10): num_of_individuals = S + 1000
else: num_of_individuals = S*300
    
total_sick = 0
    
num_of_beds = math.ceil(num_of_individuals/12)
exponential_rate = num_of_individuals/300

print(num_of_individuals, exponential_rate, num_of_beds)

1531 5.1033333333333335 128


Define the necessary set of arrays for bookkeeping

In [283]:
service_times = [] #Duration of the conversation between the customer and the operator (Service time)
daily_patient = []
daily_fullness = []

In [284]:
class Patient(object):
    def __init__(self, name, env, opr, isInitial):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.r = random.uniform(1,2)
        self.hospital2home_treating_time_rate = 6*self.r
        self.isInitial = isInitial
        self.action = env.process(self.try_to_heal())
    
    def try_to_heal(self):
        if(not self.isInitial and self.is_going_hospital() == False):
            print('%s at home at %g' % (self.name, math.ceil(self.env.now)))
            yield self.env.process(self.healing())
            print('%s is healed at %g in home' % (self.name, math.ceil(self.env.now)))
        else:
            if not self.isInitial:
              print('%s arrives hospital at %g' % (self.name, math.ceil(self.env.now)))
            with bed.request() as req:
                yield req
                if not self.isInitial:
                  print('%s is assigned to a bed at %g' % (self.name, math.ceil(self.env.now)))
                yield self.env.process(self.healing())
                print('%s is healed at %g' % (self.name, math.ceil(self.env.now)))
            
    def healing(self):
        if((self.is_going_hospital and bed.count != num_of_beds) or self.isInitial):
            duration = random.expovariate(1.0/hospital_treating_time_rate)
        elif(self.is_going_hospital):
            print("%s returns home." %(self.name))
            duration = random.expovariate(1.0/self.hospital2home_treating_time_rate)
        else:
            duration = random.expovariate(1.0/home_treating_time_rate)
        yield self.env.timeout(duration)
        service_times.append(math.ceil(duration))
        
    def is_going_hospital(self):
        random_number = random.random()
        if(random_number > 0.2): return False
        else: return True
    

In [285]:
def patient_generator(env, bed, total_sick):
    """Generate new patients with using exponential rate!!!."""
    for i in range(Total_Time):
        sick_num_in_day = round(random.expovariate(1.0/exponential_rate))
        daily_patient.append(sick_num_in_day)
        daily_fullness.append(bed.count)
        for j in range(sick_num_in_day):
            patient = Patient('Patient %s' %(total_sick+1), env, bed, False)
            total_sick = total_sick + 1
        
        yield env.timeout(1)

In [286]:
def patient_generator_half_full(env, bed, total_sick):
    for i in range (num_of_beds // 2):
        yield env.timeout(0)
        patient = Patient('Patient initial %s' %(i+1), env, bed, True)
    for i in range(Total_Time):
        sick_num_in_day = round(random.expovariate(1.0/exponential_rate))
        daily_patient.append(sick_num_in_day)
        daily_fullness.append(bed.count)

        for j in range(sick_num_in_day):
            patient = Patient('Patient %s' %(total_sick+1), env, bed, False)
            total_sick = total_sick + 1
        
        yield env.timeout(1)

In [287]:
def patient_generator_full(env, bed, total_sick):
    for i in range (num_of_beds):
        yield env.timeout(0)
        patient = Patient('Patient initial %s' %(i+1), env, bed, True)
    for i in range(Total_Time):
        sick_num_in_day = round(random.expovariate(1.0/exponential_rate)) 
        daily_patient.append(sick_num_in_day)
        daily_fullness.append(bed.count)

        for j in range(sick_num_in_day):
            patient = Patient('Patient %s' %(total_sick+1), env, bed, False)
            total_sick = total_sick + 1
        
        yield env.timeout(1)

In [288]:
env = simpy.Environment()
bed = simpy.Resource(env, capacity = num_of_beds)
env.process(patient_generator_half_full(env, bed, total_sick))
env.run(until = Total_Time)

Patient 1 arrives hospital at 0
Patient 2 at home at 0
Patient 3 at home at 0
Patient 4 at home at 0
Patient 5 at home at 0
Patient 6 at home at 0
Patient 7 arrives hospital at 0
Patient 1 is assigned to a bed at 0
Patient 7 is assigned to a bed at 0
Patient initial 62 is healed at 1
Patient initial 10 is healed at 1
Patient initial 56 is healed at 1
Patient initial 13 is healed at 1
Patient initial 28 is healed at 1
Patient initial 36 is healed at 1
Patient initial 50 is healed at 1
Patient initial 4 is healed at 1
Patient 5 is healed at 1 in home
Patient initial 21 is healed at 1
Patient initial 30 is healed at 2
Patient initial 57 is healed at 2
Patient 4 is healed at 2 in home
Patient initial 64 is healed at 2
Patient initial 12 is healed at 2
Patient initial 16 is healed at 2
Patient initial 8 is healed at 2
Patient initial 15 is healed at 2
Patient initial 60 is healed at 2
Patient initial 44 is healed at 2
Patient initial 17 is healed at 3
Patient initial 25 is healed at 3
Patie

In [289]:
print(service_times)
print(str(np.mean(service_times)))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 3, 7, 3, 3, 7, 2, 2, 7, 1, 2, 8, 3, 3, 1, 3, 8, 2, 9, 9, 9, 2, 2, 4, 10, 10, 10, 10, 1, 11, 8, 1, 1, 1, 11, 2, 4, 11, 11, 2, 2, 2, 7, 12, 12, 7, 5, 3, 3, 2, 3, 3, 3, 13, 14, 7, 2, 14, 14, 5, 10, 5, 1, 4, 1, 5, 5, 2, 6, 1, 6, 12, 11, 2, 7, 2, 3, 10, 13, 17, 3, 17, 11, 1, 13, 8, 5, 2, 19, 5, 3, 9, 5, 10, 4, 7, 14, 10, 6, 8, 17, 8, 12, 1, 12, 12, 7, 6, 16, 9, 8, 14, 5, 25, 21, 5, 10, 6, 15, 4, 5, 3, 15, 9, 12, 5, 4, 13, 3, 13, 12, 1, 15, 9, 2, 7, 15, 1, 30, 3, 9, 5, 2, 2, 26, 3, 27, 10, 9, 3, 12, 5, 4, 4, 6, 1, 3, 3, 11, 9, 9, 8, 7, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 2, 1, 1, 2, 1, 1, 9, 3, 10, 9, 3, 4, 24, 4, 13, 3, 4, 3, 1, 5, 1, 5, 7, 4, 5, 3, 7, 22, 1, 3, 3, 3, 14, 22, 1, 9, 15, 1, 11, 2, 8, 1, 8, 3, 6, 2, 3, 11, 12, 3, 3, 14, 7, 4, 10, 1, 4, 5, 3, 3, 4, 6, 1, 3, 4, 2, 7, 15, 27, 1, 7, 13, 1, 11, 2, 2, 8, 2, 22, 2, 2, 8, 4, 3, 5, 3, 3, 1, 7, 3, 4, 1,

In [290]:
print(daily_patient)
print(str(np.mean(daily_patient)))

[7, 0, 0, 1, 6, 14, 1, 12, 0, 1, 23, 8, 1, 0, 17, 7, 2, 6, 0, 1, 5, 4, 4, 5, 1, 0, 3, 5, 4, 15, 2, 1, 1, 1, 1, 17, 6, 11, 2, 1, 8, 2, 2, 18, 7, 11, 2, 3, 7, 28, 2, 4, 6, 1, 1, 2, 10, 2, 9, 0, 3, 21, 1, 2, 5, 4, 5, 4, 8, 0, 7, 2, 9, 5, 9, 4, 12, 1, 28, 0, 1, 3, 4, 5, 2, 2, 23, 10, 9, 10, 0, 5, 7, 2, 2, 4, 1, 0, 2, 2]
5.45


In [291]:
print(daily_fullness)
print(str(np.mean(daily_fullness)))
print(daily_fullness.count(0))

[63, 57, 48, 44, 42, 32, 31, 28, 27, 23, 20, 16, 14, 12, 9, 8, 9, 8, 10, 8, 8, 7, 7, 7, 7, 5, 3, 2, 2, 3, 3, 3, 3, 2, 2, 1, 3, 4, 5, 4, 3, 4, 3, 2, 9, 9, 10, 8, 7, 8, 9, 10, 10, 9, 6, 5, 4, 4, 4, 3, 3, 5, 5, 3, 1, 0, 0, 1, 1, 3, 2, 2, 2, 4, 3, 3, 3, 3, 1, 9, 7, 6, 4, 4, 4, 2, 2, 3, 3, 4, 3, 3, 3, 5, 3, 1, 0, 1, 1, 1]
8.31
3
